In [1]:
import requests
import praw

In [12]:
reddit = praw.Reddit(
    client_id = 'F371gJQiT9dtGYcmMUIKuA',
    client_secret = '3gn5Dn9QtUmWW5ee_TVmbXzlPo_wHg',
    user_agent = 'DSI Project',
    username = 'Ok_Conflict7553',
    password = 'thisisalongpassword123'
)

In [13]:
print(reddit.read_only)

False


In [52]:
import pandas as pd
import numpy as np
from praw.models import MoreComments #https://praw.readthedocs.io/en/latest/tutorials/comments.html#extracting-comments-with-praw

In [182]:
ask_reddit_dict = {}

In [183]:
titles = []
comments = []
for submission in reddit.subreddit('AskReddit').hot(limit = 10):
    submission.comment_sort = 'top'
    #comments = submission.comments
    i = 0
    while submission.comments[i].stickied:
        i += 1
    #ask_reddit_dict[submission.title] = comments[i].body
    titles.append(submission.title)
    comments.append(submission.comments[i].body)

ask_reddit_dict['title'] = titles
ask_reddit_dict['comment'] = comments
    

In [184]:
ask_reddit_dict

{'title': ['[Serious] What is the most wholesome behavior you find really attractive?',
  'What’s the hottest thing a woman can do while kissing?',
  'What is personal hell for you?',
  'Who do you think gained the most from the war in Ukraine?',
  "What's your comfort YouTube channel?",
  'What historical fact does no one like to talk about?',
  'What’s the scariest conspiracy theory you believe is 100% true?',
  'What are some things that “regular people” will likely never get to experience?',
  'What are some interesting things that our phones are capable of doing?',
  "How would you describe depression to someone who's never had it?"],
 'comment': ['When someone remembers a really random small detail I’ve only mentioned once',
  'Kiss me back aggressively, enthusiastically. Don’t just allow me to kiss you.',
  'Being somewhere for an extended period without a bathroom.',
  "NATO. It's been a fantastic logistics exercise with the side benefit of getting rid of old ammo and equipment

In [185]:
pd.DataFrame(ask_reddit_dict)

,title,comment
0,[Serious] What is the most wholesome behavior ...,When someone remembers a really random small d...
1,What’s the hottest thing a woman can do while ...,"Kiss me back aggressively, enthusiastically. D..."
2,What is personal hell for you?,Being somewhere for an extended period without...
3,Who do you think gained the most from the war ...,NATO. It's been a fantastic logistics exercise...
4,What's your comfort YouTube channel?,Tasting History with Max Miller. He does the r...
5,What historical fact does no one like to talk ...,"Native American tribes like the Cherokee, Chic..."
6,What’s the scariest conspiracy theory you beli...,A small one:\n\nSamsung announced that the Gal...
7,What are some things that “regular people” wil...,Going to space
8,What are some interesting things that our phon...,You can borrow ebooks with your existing libra...
9,How would you describe depression to someone w...,For me it was fatigue that was so intense I wo...
